In [ ]:
import matplotlib.pyplot as plt
import ROOT 
import numpy as np
import os,copy
import itertools as itrTools

In [ ]:
# kWhite  = 0,   kBlack  = 1,   kGray    = 920,  kRed    = 632,  kGreen  = 416,
# kBlue   = 600, kYellow = 400, kMagenta = 616,  kCyan   = 432,  kOrange = 800,
# kSpring = 820, kTeal   = 840, kAzure   =  860, kViolet = 880,  kPink   = 900

In [ ]:
import ROOT 
import operator 
import array

def getDefaultCanvas():
    
    defaultCanvasParams=dict({"CMS":{},"TAG":{},"EnLumi":{}})
    defaultCanvasParams["CMS"]["text"]         = "CMS"
    defaultCanvasParams["CMS"]["textSize"]     = 0.05
    defaultCanvasParams["CMS"]["textFont"]     = 61   
    defaultCanvasParams["CMS"]["xpos"]         = 0.15   
    defaultCanvasParams["CMS"]["ypos"]         = 0.98+0.004
    
    # for the "preliminary"
    defaultCanvasParams["TAG"]["text"]         = "internal"
    defaultCanvasParams["TAG"]["textSize"]     = 0.76*defaultCanvasParams["CMS"]["textSize"]  
    defaultCanvasParams["TAG"]["textFont"]     = 52   
    defaultCanvasParams["TAG"]["xpos"]         = 0.15  + 0.12
    defaultCanvasParams["TAG"]["ypos"]         = 0.98  - 0.004
    
    # for the "2018A [ 1 fb^{-1}] 13 TeV" label
    defaultCanvasParams["EnLumi"]["text"]       = "               13 TeV"
    defaultCanvasParams["EnLumi"]["textSize"]  = 0.76*defaultCanvasParams["CMS"]["textSize"]  
    defaultCanvasParams["EnLumi"]["textFont"]  = 42   
    defaultCanvasParams["EnLumi"]["xpos"]      = 0.85   
    defaultCanvasParams["EnLumi"]["ypos"]      = 0.95

    return defaultCanvasParams


def getCanvas2018A():
    cPars=getDefaultCanvas()
 #   cPars["EnLumi"]["text"]="   13 TeV"
    return cPars

def getCanvasParams(tag=None):
    cPars=getDefaultCanvas()
    print("Canvas Tag : ", tag )
    if tag=="2018Full":
        cPars["EnLumi"]["xpos"]       =  0.961
        cPars["EnLumi"]["text"]       = "SingleEG 2018 (58.9fb^{-1}) 13 TeV"
        return cPars
    if tag=="Run3MC":
        cPars["EnLumi"]["xpos"]       =  0.961
        cPars["EnLumi"]["text"]       = "Run 3 MC, 13.6 TeV"
        return cPars
    if tag=="Run3_0p900":
        cPars["EnLumi"]["xpos"]       =  0.961
        cPars["EnLumi"]["text"]       = "Run 3,900 GeV"
        return cPars
    if tag!=None:
        cPars["EnLumi"]["xpos"]       =  0.961
        cPars["EnLumi"]["text"]       =  tag
          
    return cPars

def getFitParams(hist0):
    mean  = hist0.GetMean()
    stdDev= hist0.GetStdDev()
    minX=mean-2*stdDev
    maxX=mean+2*stdDev
    hist0.Fit("gaus","Q0","",minX,maxX);
    f=hist0.GetFunction("gaus");
    return [f.GetParameter(1),f.GetParameter(2)]

def saveTheDictionary(aCollection,fname=None,folder=None):
    outfile=folder
    if fname!=None and outfile==None:
        outfile=ROOT.TFile(fname,"RECREATE")
    outfile.cd()
    for key in aCollection:
#         print("Doing : ",key)
        if type(aCollection[key])==type({}):
            print("\tMakdig dir : ",key)
            inFolder=outfile.mkdir(key)
            saveTheDictionary(aCollection[key],None,inFolder)
        else:
            aCollection[key].Write()
    if fname!=None:
        outfile.Close()

        
def getTheObjectsFromFile(aFile):
    histStore={}
    for key in aFile.GetListOfKeys():
        kk=key.GetName()
        curObj=aFile.Get(kk)
        if curObj.Class_Name()=='TDirectoryFile':
            histStore[kk]=getTheObjectsFromFile(curObj)
        else:
            histStore[kk]=curObj       
    return histStore

colurWheel=[ 
	   ROOT.kBlack  ,
	   ROOT.kRed    ,
	   ROOT.kGreen  ,
       ROOT.kBlue   ,
	   ROOT.kMagenta ,
	   ROOT.kYellow ,
	   ROOT.kOrange ,
       ROOT.kSpring ,
	   ROOT.kCyan   ,
	   ROOT.kAzure   ,
	   ROOT.kTeal   ,
	   ROOT.kGray    ,
	   ROOT.kViolet ,
	   ROOT.kPink   ,
    ]

class Plot:
    def __init__(self, **args):
        self.name        = args.get("Name", "plot")
        self.legend      = args.get("Legend",None)
        self.histo       = args.get("Histo", None)
        self.fit         = args.get("Fit", None)
        self.markerColor = args.get("MarkerColor", ROOT.kBlue+2)
        self.markerStyle = args.get("MarkerStyle", 22)
        self.markerSize = args.get("MarkerSize", 5)
        self.lineColor   = args.get("LineColor", ROOT.kBlue+2)
        self.lineStyle   = args.get("LineStyle", 1)
        self.lineWidth   = args.get("LineWidth", 1)
        self.options   = args.get("Options", "")
        self.drawLegend   = args.get("DrawLegend",False)
        self.drawLine     = args.get("DrawLine", False)
        self.histo.SetName(self.name+"_histo")
        self.normalize =args.get("Normalize",False)
        self.scaleFactor =args.get("ScaleFactor",None)
        self.doFit =args.get("doFit",False)
        self.isTH1 = args.get("isTH1",True)

class PlotCanvas:
    def __init__(self, **args):
        self.name  = ""
        self.plots = []
        self.rates = []
        self.plotDir =  args.get("prefix","plots/")
        self.yRange = (0.0, 1.0)
        self.xRange = (0, 100)
        self.yRange2 = (0.0, 1.0)
        self.xTitle = args.get("xTitle","E_{T}^{e offl} [GeV]")
        self.yTitle = args.get("yTitle","a.u")
        self.yTitle2 = args.get("yTitle","a.u")
        self.legendPosition = (0.4,0.2,0.9,0.6)
        self.descPosition = (0.6,0.58)
        self.desc = args.get("desc",["L1 Efficiency"])
        self.logx = args.get("logx",False)
        self.logy = args.get("logy",False)
        self.logy2 = args.get("logy2",False)
        self.canvasParams = args.get("canvasPars",getDefaultCanvas())    
        self.setPlotStyle()

    def addRate(self, rate):
        self.rates.append(rate)
    def addPlot(self, plot):
        self.plots.append(plot)
    def clearPlots(self):
        self.plots=[]


    def plot(self):
        xmax=0.0
        xmin=1e9
        xmax_=1e9
        xmin_=0.0
        ymax=0.0
        ymin=1e9
        ymax_=1e9
        ymin_=0.0
        for plot in self.plots:
            histo = plot.histo
            print("Scale Factor  : before  = ",plot.scaleFactor)    
            if plot.normalize:
                if plot.scaleFactor==None:
                    plot.scaleFactor=1.0/histo.Integral()
                else:
                    plot.scaleFactor/=histo.Integral()
                print("Integral = = ",1.0/histo.Integral())    
            if plot.scaleFactor!=None:
                histo.Scale(plot.scaleFactor)
            print("Scaled the histogram by a factor of : ",plot.scaleFactor)
            print("             Integral after Scaling : ",histo.Integral())
            
            if self.yRange[1]=='auto':
                ymax_=histo.GetMaximum()
                if ymax < ymax_*1.1:
                    ymax=ymax_*1.1
            if self.yRange[0]=='auto': 
                ymin_=histo.GetMinimum()
                if ymin > ymin_*0.8:
                    ymin=ymin_*0.8
            
            if self.xRange[1]=='auto':
                xmax_=histo.GetXaxis().GetBinCenter( histo.GetNbinsX() ) + 0.5*histo.GetBinWidth(histo.GetNbinsX())
                print(xmax_,xmax)
                if abs(xmax) < abs(xmax_):
                    xmax=xmax_
            if self.xRange[0]=='auto': 
                xmin_=histo.GetXaxis().GetBinCenter(1) - 0.5*histo.GetBinWidth(1)
                print(xmin_,xmin)
                if abs(xmin) > abs(xmin_):
                    xmin=xmin_
        
        if self.yRange[0]=='auto':
            self.yRange=(ymin,self.yRange[1])
        if self.yRange[1]=='auto':
            self.yRange=(self.yRange[0],ymax)
        if self.xRange[0]=='auto':
            self.xRange=(xmin,self.xRange[1])
        if self.xRange[1]=='auto':
            self.xRange=(self.xRange[0],xmax)

#         print("Setting  X-RANGE  : ",self.xRange)
#         print("Setting  Y-RANGE  : ",self.yRange)
        
#         print("xmax xmin : ",xmin,xmax)
        canvas = ROOT.TCanvas("c_"+self.name, self.name, 700, 800)
        canvas.SetGrid()
        hDummy = ROOT.TH1F("hDummy_"+self.name, self.name, 1, self.xRange[0],self.xRange[1])

        #print("ymin_,ymax_",ymin_,ymax_,"ymin , ymax - > " ,ymin,ymax)
            #print( "\tY : Min - > Max  : " ,histo.GetMinimum()," -> ", histo.GetMaximum())
        hDummy.SetAxisRange(self.yRange[0], self.yRange[1], "Y")
        hDummy.SetXTitle(self.xTitle)
        hDummy.SetYTitle(self.yTitle)
        hDummy.Draw("C")
        if self.logx : canvas.SetLogx()
        if self.logy : canvas.SetLogy()
        self.logz= True
        if self.logz : canvas.SetLogz()
        
        
        CMSbox=self.getCMSBox()
        extraTextBox=self.getExtraTextBox()
        lumibox=self.getLumiBox()


        selbox=[]
        n=0
        for inx in range(len(self.desc)):
            selbox.append(ROOT.TLatex  (self.descPosition[0], self.descPosition[1] +n*0.04, self.desc[n]))
            selbox[-1].SetNDC()
            selbox[-1].SetTextSize(0.035)
            selbox[-1].SetTextFont(12)
            selbox[-1].SetTextAlign(13)
            n+=1
        for plot in self.plots:
            if(plot.doFit):
                histo = plot.histo
                fitP  = getFitParams(histo)
                strFit='('+'{0:0.3f}'.format(fitP[0])+ ' , '+'{0:0.3f}'.format(fitP[1]) +')'
                selbox.append(ROOT.TLatex  (self.descPosition[0], self.descPosition[1] -n*0.04, strFit))
                selbox[-1].SetNDC()
                selbox[-1].SetTextSize(0.035)
                selbox[-1].SetTextFont(12)
                selbox[-1].SetTextColor(plot.lineColor)
                selbox[-1].SetTextAlign(13)
                n+=1

        #Line legend
        legend = ROOT.TLegend(self.legendPosition[0],self.legendPosition[1],self.legendPosition[2],self.legendPosition[3])
        legend.SetTextFont(42)
        legend.SetFillColor(0)

        for plot in self.plots:
            histo = plot.histo
            print("Ploting hist : ",histo.GetName())
            if(plot.isTH1):
                print( "\tX : Min - > Max  : " ,histo.GetBinCenter(1)," -> ", histo.GetBinCenter(histo.GetNbinsX())," [ ",histo.GetBinCenter(2)-histo.GetBinCenter(1)," ]" )
                print( "\tY : Min - > Max  : " ,histo.GetMinimum()," -> ", histo.GetMaximum())
                
            histo.SetMarkerStyle(plot.markerStyle)
            histo.SetMarkerSize(plot.markerSize)
            histo.SetMarkerColor(plot.markerColor)
            histo.SetLineStyle(plot.lineStyle)
            histo.SetLineWidth(plot.lineWidth)
            histo.SetLineColor(plot.lineColor)
            #histo.Draw("same e")
#             print(plot.options)
            histo.Draw("same "+plot.options)
#             print( histo.GetOption())

            if(plot.legend):
                legend.AddEntry(histo, plot.legend, "pe")
                legend.SetTextSize(0.025) 
            if(plot.drawLegend) :  legend.Draw()
        
        scale = (self.yRange[1] -self.yRange[0])/(self.yRange2[1] - self.yRange2[0])
        y20=self.yRange2[0]

        if(self.logy2):
            scale = (self.yRange[1] -self.yRange[0])/(ROOT.log(self.yRange2[1]) - ROOT.log(self.yRange2[0]) )
            y20 = ROOT.log(self.yRange2[0])


        #####   begin :            For plotting a graph in the Second Axis         #####

        if len(self.rates) >0 :
            axis=None
            canvas.SetRightMargin(0.18);
            if(self.logy2):
                axis = ROOT.TGaxis(self.xRange[1],self.yRange[0],self.xRange[1], self.yRange[1],self.yRange2[0],self.yRange2[1],510,"+LG")
            else :
                axis = ROOT.TGaxis(self.xRange[1],self.yRange[0],self.xRange[1], self.yRange[1],self.yRange2[0],self.yRange2[1],510,"+L")
            axis.SetLineColor(ROOT.kRed);
            axis.SetTextColor(ROOT.kRed);
            axis.SetTitle(self.yTitle2)
            axis.SetTitleColor(ROOT.kBlack)
            axis.SetTitleOffset(1.2)
            axis.Draw();

        for rate in self.rates:
            histo = rate.histo
            if self.logy2:
                for i in range(histo.GetNbinsX()):
                    content=histo.GetBinContent(i+1)
                    scaledContent = ( ROOT.log(content + 1e-20 )  - y20 )* scale
                    histo.SetBinContent(i+1,scaledContent)
                    #print("\t ",content," -> ",scaledContent,"  = ( ",ROOT.log(content + 1e-20),"  - ",y20," )* ",scale)
            else:
                for i in range(histo.GetNbinsX()):
                    content=histo.GetBinContent(i+1)
                    scaledContent = (content  - y20 )* scale
                    histo.SetBinContent(i+1,scaledContent)
            
            histo.SetMarkerStyle(rate.markerStyle)
            histo.SetMarkerColor(rate.markerColor)
            histo.SetLineColor(rate.markerColor)
            histo.Draw("pl same "+ rate.options)
            legend.AddEntry(histo, rate.legend, "pe")
            legend.SetTextSize(0.025) 
            if(rate.drawLegend) :  legend.Draw()
        
        #####   end :            For plotting a graph in the Second Axis         #####
            
        CMSbox.Draw()
        extraTextBox.Draw()
        lumibox.Draw()
        for selb in selbox:
            selb.Draw()
        
#         canvas.Print(self.plotDir+"/"+self.name+".pdf", "pdf")
        canvas.Print(self.plotDir+"/"+self.name+".png", "png")

        return canvas

    def getCMSBox(self):
        par=self.canvasParams["CMS"]
        CMSbox       = ROOT.TLatex  (par["xpos"],par["ypos"], par["text"])
        CMSbox.SetNDC()
        CMSbox.SetTextSize(par["textSize"])
        CMSbox.SetTextFont(par["textFont"])
        CMSbox.SetTextColor(ROOT.kBlack)
        CMSbox.SetTextAlign(13) 
        return CMSbox
   
    def getExtraTextBox(self):
        par=self.canvasParams["TAG"]
        extraTextBox = ROOT.TLatex  (par["xpos"], par["ypos"], par["text"])
        extraTextBox.SetNDC()
        extraTextBox.SetTextSize(par["textSize"])
        extraTextBox.SetTextFont(par["textFont"])
        extraTextBox.SetTextColor(ROOT.kBlack)
        extraTextBox.SetTextAlign(13)
        
        return extraTextBox

    def getLumiBox(self):
        par=self.canvasParams["EnLumi"]
        lumibox = ROOT.TLatex  ( par["xpos"], par["ypos"], par["text"])
        lumibox.SetNDC()
        lumibox.SetTextAlign(31)
        lumibox.SetTextSize(par["textSize"])
        lumibox.SetTextFont(par["textFont"])
        lumibox.SetTextColor(ROOT.kBlack)
        
        return lumibox
        
    def setPlotStyle(self):
        ROOT.gROOT.SetStyle("Plain")
        ROOT.gStyle.SetOptStat()
        ROOT.gStyle.SetOptFit(0)
        ROOT.gStyle.SetOptTitle(0)
        ROOT.gStyle.SetFrameLineWidth(2)
        ROOT.gStyle.SetPadBottomMargin(0.13)
        ROOT.gStyle.SetPadLeftMargin(0.15)
        ROOT.gStyle.SetPadTopMargin(0.06)
        ROOT.gStyle.SetPadRightMargin(0.05)

        ROOT.gStyle.SetLabelFont(42,"X")
        ROOT.gStyle.SetLabelFont(42,"Y")
        ROOT.gStyle.SetLabelSize(0.04,"X")
        ROOT.gStyle.SetLabelSize(0.04,"Y")
        ROOT.gStyle.SetLabelOffset(0.01,"Y")
        ROOT.gStyle.SetTickLength(0.02,"X")
        ROOT.gStyle.SetTickLength(0.02,"Y")
        ROOT.gStyle.SetLineWidth(2)
        ROOT.gStyle.SetTickLength(0.02 ,"Z")

        ROOT.gStyle.SetTitleSize(0.1)
        ROOT.gStyle.SetTitleFont(42,"X")
        ROOT.gStyle.SetTitleFont(42,"Y")
        ROOT.gStyle.SetTitleSize(0.05,"X")
        ROOT.gStyle.SetTitleSize(0.05,"Y")
        ROOT.gStyle.SetTitleOffset(1.1,"X")
        ROOT.gStyle.SetTitleOffset(1.4,"Y")
        ROOT.gStyle.SetOptStat(0)
        ROOT.gStyle.SetPalette(1)
        ROOT.gStyle.SetPaintTextFormat("3.2f")
        ROOT.gROOT.ForceStyle()

col=[ 0,1,2,ROOT.kGreen+1,4,6,ROOT.kOrange+2,ROOT.kMagenta+2,9,10,11,12,13,14,15  ]

def getDefaultPlotParams(col=1,marker=22,markerSize=1):
    return {'Legend':"Data : 2018, Unpacked",
                          'MarkerColor' : col,
                          'LineColor':col,
                          'LineWidth':2,
                          'LineStyle':9,
                          'MarkerStyle':marker,
                          'MarkerSize':markerSize,
                          'DrawLegend':True,
                          'Option':'pc',
                           'isMC' : True
            } 

def getDefaultPlot(prefix='plots/',name='defaultName',cPars=getCanvasParams('GEN')):
#     plot=putil.PlotCanvas(prefix=prefix,canvasPars=cPars)
    plot=PlotCanvas(prefix=prefix,canvasPars=cPars)
    
    plot.name   = name
    plot.xRange = (3.0,50.0)  
    plot.yRange = (0.0,1.02) 
    plot.yTitle = "#epsilon"  
    plot.xTitle = "E_{T}^{offl.}"
    plot.desc   = ["L1 EG Isolation"] 
    plot.legendPosition = (0.6,0.5,0.90,0.65)
    plot.descPosition   = (0.6,0.75)     
    plot.logx = False
    plot.logy = False
    return plot

### GenPlots

In [ ]:
! mkdir results/plots -p

pltPrefix='results/plots/gridV4_WithPFA1p_v2prime/'

if not os.path.exists(pltPrefix):
    os.system("mkdir -p "+pltPrefix)

In [ ]:
Run2RateFile='data/HistgramFile_Rate_Run3MC_2022v0_2_recalibV1.root'
Run2TurnOnFile='data/run3MC_122X_v1p0_2022_v0_2_turnon.root'
Run2TurnOnFile='data/run3MC_122X_v1p0_2022_v0_2_turnon.root'
Run2TurnOnFile='HistgramFile_Run3MC_withPFAp.root'

In [ ]:
_fileRun2TurnOns=ROOT.TFile(Run2TurnOnFile)
fileRun2TurnOns=_fileRun2TurnOns.Get('TGraphs')
Run2TurnOnMap={}
Run2TurnOnMap={}
Run2TightTurnOnMap={}
Run2LooseTurnOnMap={}
for ky in  fileRun2TurnOns.GetListOfKeys():
    hName=ky.GetName()
    if 'Efficiency' not in hName:
        continue
    if 'tight' in hName:
        continue
    if 'loose' in hName:
        continue
    if 'def' in hName:
        continue
    if 'L1Et' not in hName:
        continue
    eT=int(hName.replace('hist_L1Et','').replace('Efficiency',''))
    Run2TurnOnMap[eT]=fileRun2TurnOns.Get(hName)
    if eT%10==0:
        print('For ',eT,' Adding ',hName)
        
        
for ky in  fileRun2TurnOns.GetListOfKeys():
    hName=ky.GetName()
    if 'Efficiency' not in hName:
        continue
    if 'tight' not in hName:
        continue
    if 'L1Et' not in hName:
        continue
    if 'def' in hName:
        continue
#     print(hName)
    eT=int(hName.replace('hist_L1Et','').replace('_tightisoEfficiency',''))
    Run2TightTurnOnMap[eT]=fileRun2TurnOns.Get(hName)
    if eT%10==0:
        print('For ',eT,' Adding ',hName)

for ky in  fileRun2TurnOns.GetListOfKeys():
    hName=ky.GetName()
    if 'Efficiency' not in hName:
        continue
    if 'loose' not in hName:
        continue
    if 'L1Et' not in hName:
        continue
    if 'def' in hName:
        continue
#     print(hName)
    eT=int(hName.replace('hist_L1Et','').replace('_looseisoEfficiency',''))
    Run2LooseTurnOnMap[eT]=fileRun2TurnOns.Get(hName)
    if eT%10==0:
        print('For ',eT,' Adding ',hName)

In [ ]:
_fileRun2TurnOns=ROOT.TFile(Run2TurnOnFile)
fileRun2TurnOns=_fileRun2TurnOns.Get('TGraphs')
Run2TurnOnMap={}
Run2TightTurnOnMap={}
Run2LooseTurnOnMap={}
print("Inclusive")   
for ky in  fileRun2TurnOns.GetListOfKeys():
    hName=ky.GetName()
    if 'divide' not in hName:
        continue
    if 'Tight' in hName:
        continue
    if 'Loose' in hName:
        continue
    print(hName)
    eT=int(hName.split('EG')[1].split('_')[0])
    Run2TurnOnMap[eT]=fileRun2TurnOns.Get(hName)
    if eT%10==0:
        print('For ',eT,' Adding ',hName)
        

print("Tight")        
for ky in  fileRun2TurnOns.GetListOfKeys():
    hName=ky.GetName()

    if 'Tight' not in hName:
        continue
    eT=int(hName.split('EG')[1].split('_')[0])
    Run2LooseTurnOnMap[eT]=fileRun2TurnOns.Get(hName)
    eT=int(hName.split('EG')[1].split('_')[0])
    Run2TightTurnOnMap[eT]=fileRun2TurnOns.Get(hName)
    if eT%10==0:
        print('For ',eT,' Adding ',hName)
print("Loose")
for ky in  fileRun2TurnOns.GetListOfKeys():
    hName=ky.GetName()

    if 'Loose' not in hName:
        continue
    eT=int(hName.split('EG')[1].split('_')[0])
    Run2LooseTurnOnMap[eT]=fileRun2TurnOns.Get(hName)
    if eT%10==0:
        print('For ',eT,' Adding ',hName)

## Isolation Scans Pick And Plot

    For checking and identifying if the best turn ons are okay at low ets 

In [ ]:
fStore={}
# fStore['pl143']=ROOT.TFile('workarea/gridV4_122X_withPFA1p/HistgramFile_step3step4_12XnewLayer1_run3PFA1p_update_v1p16.root')
# fStore['pl405']=ROOT.TFile('workarea/gridV4_122X_withPFA1p/HistgramFile_step3step4_12XnewLayer1_run3PFA1p_update_v1p45.root')
# fStore['pl407']=ROOT.TFile('workarea/gridV4_122X_withPFA1p/HistgramFile_step3step4_12XnewLayer1_run3PFA1p_update_v1p46.root')
# fStore['pl142']=ROOT.TFile('workarea/gridV4_122X_withPFA1p/HistgramFile_step3step4_12XnewLayer1_run3PFA1p_update_v1p16.root')
# fStore['pl926']=ROOT.TFile('workarea/gridV4_122X_withPFA1p/HistgramFile_step3step4_12XnewLayer1_run3PFA1p_update_v1p103.root')

fStore['pl153']=ROOT.TFile('workarea/gridV4_122X_withPFA1p/HistogramFile_step3step4_withPFApp16.root')
fStore['pl716']=ROOT.TFile('workarea/gridV4_122X_withPFA1p/HistogramFile_step3step4_withPFApp72.root')


hStore={}
for ky in fStore :
    hStore[ky]=getTheObjectsFromFile(fStore[ky])

## Efficiency plots

In [ ]:
histsToPlot={}
fTag='pl153'
opt='153'
eTs_toget=[20,22,25,28,30,32,34]
histsToPlot['Option '+opt]={}
for hName in hStore[fTag]['turn_on_progression'].keys():
    if opt not in hName:
        continue
#     eT=int(hName.split('_')[8])
    eT=int(hName.split('_')[7].replace('Et',''))
    if eT not in eTs_toget:
        continue
    print("Got ",hName)
    histsToPlot['Option '+opt][eT]=hStore[fTag]['turn_on_progression'][hName]
print()

histsToPlot['Loose Iso. Run3 Prelim' ]=histsToPlot.pop('Option '+opt)

In [ ]:
fTag='pl716'
opt='716'
histsToPlot['Option '+opt]={}
for hName in hStore[fTag]['turn_on_progression'].keys():
    if 'on'+opt+'_' not in hName:
        continue
#     eT=int(hName.split('_')[8])
    eT=int(hName.split('_')[7].replace('Et',''))
    if eT not in eTs_toget:
        continue
    print("Got ",hName)
    histsToPlot['Option '+opt][eT]=hStore[fTag]['turn_on_progression'][hName]
print()
histsToPlot['Tight Iso. Run3 Prelim' ]=histsToPlot.pop('Option '+opt)

In [ ]:
if False:  #  Plots for diff eTs , inclusive / tight and loose in same one
    plots=[]
    for eT in [15,18,20]:
        cPars=getCanvasParams('Run3MC')
        plot=getDefaultPlot(prefix=pltPrefix+'/macroscopic/',name='bkpE'+str(eT)+'_AllEffficiencies',cPars=cPars) 
        plot.legendPosition = (0.65,0.37,0.94,0.55)
        plot.descPosition   = (0.65,0.60)
        plot.desc =['#bf{Run III Prelims}','L1 EG']
        plot.yTitle = "#epsilon"  
        plot.xTitle = "E_{T}^{offline}"
        plot.xRange = (12.0,40.0) 
    #         plot.yRange = (1e-3,yMax[hName]*1e1) 
        plot.yRange = (0.80,1.02) 
        plot.logx = False
        plot.logy = False
        plots.append(plot)

        i=1
        print("Et : ",eT)
        hist=ROOT.TGraphAsymmErrors(Run2TurnOnMap[eT]).Clone()
        pparams=getDefaultPlotParams(col=i,marker=3)
        pparams['Legend']='Inclusive'+" E_{T} > "+str(eT)
        pparams['MarkerColor']= col[i]; pparams['LineColor'] = col[i];
        pparams['LineSize'] = 2
        pparams['Options']='l'
        pparams['isTH1']=False
        aplot = Plot(Name=hist.GetName(), Histo=hist,**pparams)
        aplot.normalize = False ;aplot.scaleFactor = None
        plots[-1].addPlot(aplot)

    #     i+=1
        tag='Tight Iso. Run3 Prelim'   
        print("\t",tag)
        hist=histsToPlot[tag][eT].Clone()
        hName=hist.GetName()
        pparams=getDefaultPlotParams(col=i,marker=3)
        pparams['Legend']="Tight Iso ,"+" E_{T} > "+str(eT)
        c=4
        pparams['MarkerColor']= c; pparams['LineColor'] = c;
        pparams['LineSize'] = 2
        pparams['Options']='l'
        pparams['isTH1']=False
        aplot = Plot(Name=hist.GetName(), Histo=hist,**pparams)
        aplot.normalize = False ;aplot.scaleFactor = None
#             aplot.normalize = True ;aplot.scaleFactor = 1.0*xSections[tag]
        plots[-1].addPlot(aplot)


        tag='Loose Iso. Run3 Prelim'   
        print("\t",tag)
        i+=1
        hist=histsToPlot[tag][eT].Clone()
        hName=hist.GetName()
        pparams=getDefaultPlotParams(col=i,marker=3)
        pparams['Legend']="Loose Iso. "+", E_{T} > "+str(eT)
        c=2
        pparams['MarkerColor']= c; pparams['LineColor'] = c;
        pparams['LineSize'] = 2
        pparams['Options']='l'
        pparams['isTH1']=False
        aplot = Plot(Name=hist.GetName(), Histo=hist,**pparams)
        aplot.normalize = False ;aplot.scaleFactor = None
#             aplot.normalize = True ;aplot.scaleFactor = 1.0*xSections[tag]
        plots[-1].addPlot(aplot)

   
    canvas = []
    for plot in plots:
        canvas.append(plot.plot())             

In [ ]:

col=[ 0,1,2,ROOT.kGreen+1,4,6,ROOT.kOrange+2,ROOT.kMagenta+2,9,10,11,12,13,14,15  ]


if True:  #  Plots for diff eTs , inclusive / tight and loose in same one
    plots=[]
    for eT in eTs_toget:
        cPars=getCanvasParams('Run3MC')
        plot=getDefaultPlot(prefix=pltPrefix,name='E'+str(eT)+'_AllEffficiencies',cPars=cPars) 
        plot.legendPosition = (0.65,0.57,0.94,0.75)
        plot.descPosition   = (0.65,0.80)
        plot.desc =['#bf{Run III Prelims}','L1 EG']
        plot.yTitle = "#epsilon"  
        plot.xTitle = "E_{T}^{offline}"
        plot.xRange = (8.0,100.0) 
    #         plot.yRange = (1e-3,yMax[hName]*1e1) 
        plot.yRange = (0.0,1.02) 
        plot.logx = True
        plot.logy = False
        plots.append(plot)

        i=1
        print("Et : ",eT)
        hist=ROOT.TGraphAsymmErrors(Run2TurnOnMap[eT]).Clone()
        pparams=getDefaultPlotParams(col=i,marker=3)
        pparams['Legend']='Inclusive'+" E_{T} > "+str(eT)
        pparams['MarkerColor']= col[i]; pparams['LineColor'] = col[i];
        pparams['LineSize'] = 2
        pparams['Options']='l'
        pparams['isTH1']=False
        aplot = Plot(Name=hist.GetName(), Histo=hist,**pparams)
        aplot.normalize = False ;aplot.scaleFactor = None
        plots[-1].addPlot(aplot)

    #     i+=1
        tag='Tight Iso. Run3 Prelim'   
        print("\t",tag)
        hist=histsToPlot[tag][eT].Clone()
        hName=hist.GetName()
        pparams=getDefaultPlotParams(col=i,marker=3)
        pparams['Legend']="Tight Iso ,"+" E_{T} > "+str(eT)
        c=4
        pparams['MarkerColor']= c; pparams['LineColor'] = c;
        pparams['LineSize'] = 2
        pparams['Options']='l'
        pparams['isTH1']=False
        aplot = Plot(Name=hist.GetName(), Histo=hist,**pparams)
        aplot.normalize = False ;aplot.scaleFactor = None
#             aplot.normalize = True ;aplot.scaleFactor = 1.0*xSections[tag]
        plots[-1].addPlot(aplot)


        tag='Loose Iso. Run3 Prelim'   
        print("\t",tag)
        i+=1
        hist=histsToPlot[tag][eT].Clone()
        hName=hist.GetName()
        pparams=getDefaultPlotParams(col=i,marker=3)
        pparams['Legend']="Loose Iso. "+", E_{T} > "+str(eT)
        c=2
        pparams['MarkerColor']= c; pparams['LineColor'] = c;
        pparams['LineSize'] = 2
        pparams['Options']='l'
        pparams['isTH1']=False
        aplot = Plot(Name=hist.GetName(), Histo=hist,**pparams)
        aplot.normalize = False ;aplot.scaleFactor = None
#             aplot.normalize = True ;aplot.scaleFactor = 1.0*xSections[tag]
        plots[-1].addPlot(aplot)

   
    canvas = []
    for plot in plots:
        canvas.append(plot.plot())             

In [ ]:
Run2TurnOnMap

### Rate plots

### Rate plots

In [ ]:
fileRate=ROOT.TFile(Run2RateFile)

Run2RateHist_SingleEG_rate_TightIso=fileRate.Get('SingleEG_rate_TightIso')
Run2RateHist_DoubleEG_rate_LooseIso=fileRate.Get('DoubleEG_rate_LooseIso')
Run2RateHist_SingleEG_rate_inclusive=fileRate.Get('SingleEG_rate_inclusive')

In [ ]:
Run3DefaultRateFile='data/HistgramFile_Rate_Run3MC_Run3IsoLUT_default.root'
fileDefRun3Rate=ROOT.TFile(Run3DefaultRateFile)
Run3DeafultRateHist_SingleEG_rate_TightIso=fileDefRun3Rate.Get('SingleEG_rate_TightIso')
Run3DeafultRateHist_DoubleEG_rate_LooseIso=fileDefRun3Rate.Get('DoubleEG_rate_LooseIso')
Run3DeafultRateHist_SingleEG_rate_inclusive=fileDefRun3Rate.Get('SingleEG_rate_inclusive')

In [ ]:
fTag_tight='pl716'
opt_tight='716'

fTag_loose='pl153'
opt_loose='153'

In [ ]:
ratesToPlot={}
fTag=fTag_tight
opt=opt_tight
for hName in hStore[fTag]['rate_progression'].keys():
    if opt not in hName:
        continue
    print("Got ",hName)
    ratesToPlot['Option '+opt]=hStore[fTag]['rate_progression'][hName]
print()

ratesToPlot['Tight Iso. Run3 Prelim' ]=ratesToPlot.pop('Option '+opt)

fTag=fTag_loose
opt=opt_loose
ratesToPlot['Option '+opt]={}
for hName in hStore[fTag]['rate_progression_double'].keys():
    if 'double_'+opt+'_' not in hName:
        continue
    print("Got ",hName)
    ratesToPlot['Option '+opt]=hStore[fTag]['rate_progression_double'][hName]
print()
ratesToPlot['Loose Iso. Run3 Prelim' ]=ratesToPlot.pop('Option '+opt)

#### Rates

In [ ]:
if True:  # Rates Isclusive / Isolated Tight / Isolated Loose
    plots=[]
    cPars=getCanvasParams('Run3MC')
    plot=getDefaultPlot(prefix=pltPrefix,name='Rates_prelims',cPars=cPars) 
    plot.legendPosition = (0.55,0.55,0.9,0.75)
    plot.descPosition   = (0.60,0.80)
    plot.desc =['#bf{Run III Prelims}','L1 EG']
    plot.yTitle = "Rate  [ kHz ]"  
    plot.xTitle = "E_{T}^{L1}"
    plot.xRange = (1.0,60.0) 
    plot.yRange = (1e-1,3e4) 
    plot.logy = True
    plots.append(plot)
    
    i=1
    
    i+=1
    hist=Run2RateHist_SingleEG_rate_inclusive.Clone()
    pparams=getDefaultPlotParams(col=i,marker=3)
    pparams['Legend']='Inclusive'
    pparams['MarkerColor']= col[i]; pparams['LineColor'] = col[i];
    pparams['MarkerStyle']= 22
    pparams['LineSize'] = 2
    pparams['Options']='pec'
    pparams['isTH1']=False
    aplot = Plot(Name=hist.GetName(), Histo=hist,**pparams)
    aplot.normalize = False ;aplot.scaleFactor = None
    plots[-1].addPlot(aplot)
    
    
    i+=1
    tag='Tight Iso. Run3 Prelim'
    hist=ratesToPlot[tag].Clone()
    pparams=getDefaultPlotParams(col=i,marker=3)
    pparams['Legend']="Single EG , Tight Iso."
    pparams['MarkerColor']= col[i]; pparams['LineColor'] = col[i];
    pparams['MarkerStyle']= 22
    pparams['LineSize'] = 2
    pparams['Options']='pec'
    pparams['isTH1']=False
    aplot = Plot(Name=hist.GetName(), Histo=hist,**pparams)
    aplot.normalize = False ;aplot.scaleFactor = None
    plots[-1].addPlot(aplot)

    i+=1
    tag='Loose Iso. Run3 Prelim'
    hist=ratesToPlot[tag].Clone()
    pparams=getDefaultPlotParams(col=i,marker=3)
    pparams['Legend']="Double EG , Loose Iso."
    pparams['MarkerColor']= col[i]; pparams['LineColor'] = col[i];
    pparams['MarkerStyle']= 22
    pparams['LineSize'] = 2
    pparams['Options']='pec'
    pparams['isTH1']=False
    aplot = Plot(Name=hist.GetName(), Histo=hist,**pparams)
    aplot.normalize = False ;aplot.scaleFactor = None 
    plots[-1].addPlot(aplot)
    canvas = []
    
    for plot in plots:
        canvas.append(plot.plot())            
 

### Rate Plot Sanity check : Double EG Rates

In [ ]:
ratesToPlotV2={}
fTag=fTag_tight
opt=opt_tight
for hName in hStore[fTag]['rate_progression_double'].keys():
    if opt not in hName:
        continue
    print("Got ",hName)
    ratesToPlotV2['Option '+opt]=hStore[fTag]['rate_progression_double'][hName]
print()

ratesToPlotV2['Tight Iso. Run3 Prelim' ]=ratesToPlotV2.pop('Option '+opt)

fTag=fTag_loose
opt=opt_loose
ratesToPlotV2['Option '+opt]={}
for hName in hStore[fTag]['rate_progression_double'].keys():
    if 'double_'+opt+'_' not in hName:
        continue
    print("Got ",hName)
    ratesToPlotV2['Option '+opt]=hStore[fTag]['rate_progression_double'][hName]
print()
ratesToPlotV2['Loose Iso. Run3 Prelim' ]=ratesToPlotV2.pop('Option '+opt)

In [ ]:
if True:  # Rates Isclusive / Isolated Tight / Isolated Loose
    plots=[]
    cPars=getCanvasParams('Run3MC')
    plot=getDefaultPlot(prefix=pltPrefix,name='Rates_SanityCheckDEG_',cPars=cPars) 
    plot.legendPosition = (0.55,0.55,0.9,0.75)
    plot.descPosition   = (0.60,0.80)
    plot.desc =['#bf{Run III Prelims}','L1 EG']
    plot.yTitle = "Rate  [ kHz ]"  
    plot.xTitle = "E_{T}^{L1}"
    plot.xRange = (1.0,60.0) 
    plot.yRange = (1e-1,3e4) 
    plot.logy = True
    plots.append(plot)
    
    i=1
    
    i+=1
    hist=Run2RateHist_SingleEG_rate_inclusive.Clone()
    pparams=getDefaultPlotParams(col=i,marker=3)
    pparams['Legend']='Inclusive'
    pparams['MarkerColor']= col[i]; pparams['LineColor'] = col[i];
    pparams['MarkerStyle']= 22
    pparams['LineSize'] = 2
    pparams['Options']='pec'
    pparams['isTH1']=False
    aplot = Plot(Name=hist.GetName(), Histo=hist,**pparams)
    aplot.normalize = False ;aplot.scaleFactor = None
    plots[-1].addPlot(aplot)
    
    
    i+=1
    tag='Tight Iso. Run3 Prelim'
    hist=ratesToPlotV2[tag].Clone()
    pparams=getDefaultPlotParams(col=i,marker=3)
    pparams['Legend']="Single EG , Tight Iso."
    pparams['MarkerColor']= col[i]; pparams['LineColor'] = col[i];
    pparams['MarkerStyle']= 22
    pparams['LineSize'] = 2
    pparams['Options']='pec'
    pparams['isTH1']=False
    aplot = Plot(Name=hist.GetName(), Histo=hist,**pparams)
    aplot.normalize = False ;aplot.scaleFactor = None
    plots[-1].addPlot(aplot)

    i+=1
    tag='Loose Iso. Run3 Prelim'
    hist=ratesToPlotV2[tag].Clone()
    pparams=getDefaultPlotParams(col=i,marker=3)
    pparams['Legend']="Double EG , Loose Iso."
    pparams['MarkerColor']= col[i]; pparams['LineColor'] = col[i];
    pparams['MarkerStyle']= 22
    pparams['LineSize'] = 2
    pparams['Options']='pec'
    pparams['isTH1']=False
    aplot = Plot(Name=hist.GetName(), Histo=hist,**pparams)
    aplot.normalize = False ;aplot.scaleFactor = None 
    plots[-1].addPlot(aplot)
    canvas = []
    
    for plot in plots:
        canvas.append(plot.plot())            
 

## Rates Sanity check : Single EG

In [ ]:
ratesToPlotV2={}
fTag=fTag_tight
opt=opt_tight
for hName in hStore[fTag]['rate_progression'].keys():
    if opt not in hName:
        continue
    print("Got ",hName)
    ratesToPlotV2['Option '+opt]=hStore[fTag]['rate_progression'][hName]
print()

ratesToPlotV2['Tight Iso. Run3 Prelim' ]=ratesToPlotV2.pop('Option '+opt)

fTag=fTag_loose
opt=opt_loose
ratesToPlotV2['Option '+opt]={}
for hName in hStore[fTag]['rate_progression'].keys():
    if 'n_'+opt+'_' not in hName:
        continue
    print("Got ",hName)
    ratesToPlotV2['Option '+opt]=hStore[fTag]['rate_progression'][hName]
print()
ratesToPlotV2['Loose Iso. Run3 Prelim' ]=ratesToPlotV2.pop('Option '+opt)

In [ ]:
if True:  # Rates Isclusive / Isolated Tight / Isolated Loose
    plots=[]
    cPars=getCanvasParams('Run3MC')
    plot=getDefaultPlot(prefix=pltPrefix,name='Rates_SanityCheckSEG_',cPars=cPars) 
    plot.legendPosition = (0.55,0.55,0.9,0.75)
    plot.descPosition   = (0.60,0.80)
    plot.desc =['#bf{Run III Prelims}','L1 EG']
    plot.yTitle = "Rate  [ kHz ]"  
    plot.xTitle = "E_{T}^{L1}"
    plot.xRange = (1.0,60.0) 
    plot.yRange = (1e-1,3e4) 
    plot.logy = True
    plots.append(plot)
    
    i=1
    
    i+=1
    hist=Run2RateHist_SingleEG_rate_inclusive.Clone()
    pparams=getDefaultPlotParams(col=i,marker=3)
    pparams['Legend']='Inclusive'
    pparams['MarkerColor']= col[i]; pparams['LineColor'] = col[i];
    pparams['MarkerStyle']= 22
    pparams['LineSize'] = 2
    pparams['Options']='pec'
    pparams['isTH1']=False
    aplot = Plot(Name=hist.GetName(), Histo=hist,**pparams)
    aplot.normalize = False ;aplot.scaleFactor = None
    plots[-1].addPlot(aplot)
    
    
    i+=1
    tag='Tight Iso. Run3 Prelim'
    hist=ratesToPlotV2[tag].Clone()
    pparams=getDefaultPlotParams(col=i,marker=3)
    pparams['Legend']="Single EG , Tight Iso."
    pparams['MarkerColor']= col[i]; pparams['LineColor'] = col[i];
    pparams['MarkerStyle']= 22
    pparams['LineSize'] = 2
    pparams['Options']='pec'
    pparams['isTH1']=False
    aplot = Plot(Name=hist.GetName(), Histo=hist,**pparams)
    aplot.normalize = False ;aplot.scaleFactor = None
    plots[-1].addPlot(aplot)

    i+=1
    tag='Loose Iso. Run3 Prelim'
    hist=ratesToPlotV2[tag].Clone()
    pparams=getDefaultPlotParams(col=i,marker=3)
    pparams['Legend']="Double EG , Loose Iso."
    pparams['MarkerColor']= col[i]; pparams['LineColor'] = col[i];
    pparams['MarkerStyle']= 22
    pparams['LineSize'] = 2
    pparams['Options']='pec'
    pparams['isTH1']=False
    aplot = Plot(Name=hist.GetName(), Histo=hist,**pparams)
    aplot.normalize = False ;aplot.scaleFactor = None 
    plots[-1].addPlot(aplot)
    canvas = []
    
    for plot in plots:
        canvas.append(plot.plot())            
 

## Run3 Unpacked Vs Run3 Prelims

#### Efficeincy

In [ ]:
_run3UnpackedTOnFile=ROOT.TFile('data/egamma2022BCDPostRecalib_TagAndProbe_DataRun3_2022turnon.root','READ')
run3UnpackedTOnFile=_run3UnpackedTOnFile.Get('0')
Run3InclusiveTurnOnMap={}
Run3LooseTurnOnMap={}
Run3TightTurnOnMap={}

for ky in  run3UnpackedTOnFile.GetListOfKeys():
    hName=ky.GetName()
    if 'Efficiency' not in hName:
        continue
    if 'tight' in hName:
        continue
    if 'loose' in hName:
        continue
    if 'def' in hName:
        continue
    if 'L1Et' not in hName:
        continue
    if 'Graph_from' in hName:
        continue
    eT=int(hName.replace('hist_L1Et','').replace('Efficiency',''))
    Run3InclusiveTurnOnMap[eT]=run3UnpackedTOnFile.Get(hName)
    if eT%50==0:
        print('For ',eT,' Adding ',hName)        
        
for ky in  run3UnpackedTOnFile.GetListOfKeys():
    hName=ky.GetName()
    if 'Efficiency' not in hName:
        continue
    if 'tight' not in hName:
        continue
    if 'L1Et' not in hName:
        continue
    if 'def' in hName:
        continue
    if 'Graph_from' in hName:
        continue
    if 'Barrel' in hName:
        continue
    if 'ECap' in hName:
        continue
    eT=int(hName.replace('hist_L1Et','').replace('_tightisoEfficiency',''))
    Run3TightTurnOnMap[eT]=run3UnpackedTOnFile.Get(hName)
    if eT%50==0:
        print('For ',eT,' Adding ',hName)

for ky in  run3UnpackedTOnFile.GetListOfKeys():
    hName=ky.GetName()
    if 'Efficiency' not in hName:
        continue
    if 'loose' not in hName:
        continue
    if 'L1Et' not in hName:
        continue
    if 'def' in hName:
        continue
    if 'Graph_from' in hName:
        continue
    if 'Barrel' in hName:
        continue
    if 'ECap' in hName:
        continue
#     print(hName)
    eT=int(hName.replace('hist_L1Et','').replace('_looseisoEfficiency',''))
    Run3LooseTurnOnMap[eT]=run3UnpackedTOnFile.Get(hName)
    if eT%50==0:
        print('For ',eT,' Adding ',hName)
                

In [ ]:
if True:  #  Plots for diff eTs Tight Vs Loose  Run3 Unpacked vs  Run3 Proposal
    plots=[]
    for eT in eTs_toget:
        cPars=getCanvasParams('Run3MC')
        plot=getDefaultPlot(prefix=pltPrefix,name='E'+str(eT)+'_VsRun3',cPars=cPars) 
        plot.legendPosition = (0.5,0.37,0.94,0.55)
        plot.descPosition   = (0.2,0.60)
        plot.desc =['#bf{Run III Prelims}','L1 EG','E_{T} > '+str(eT)]
        plot.yTitle = "#epsilon"  
        plot.xTitle = "E_{T}^{offline}"
        plot.xRange = (8.0,100.0) 
    #         plot.yRange = (1e-3,yMax[hName]*1e1) 
        plot.yRange = (0.0,1.02) 
        plot.logx = True
        plot.logy = False
        plots.append(plot)

        i=1
        print("Et : ",eT)
        hist=ROOT.TGraphAsymmErrors(Run3TightTurnOnMap[eT]).Clone()
        pparams=getDefaultPlotParams(col=i,marker=3)
        pparams['Legend']='Run 3 Unpacked Tight Iso. '
        col=ROOT.kBlue
        pparams['MarkerColor']=col ; pparams['LineColor'] = col;
        pparams['LineSize'] = 2
        pparams['Options']='l'
        pparams['isTH1']=False
        aplot = Plot(Name=hist.GetName(), Histo=hist,**pparams)
        aplot.normalize = False ;aplot.scaleFactor = None
        plots[-1].addPlot(aplot)

        i+=1
        print("Et : ",eT)
        hist=ROOT.TGraphAsymmErrors(Run3LooseTurnOnMap[eT]).Clone()
        pparams=getDefaultPlotParams(col=i,marker=3)
        pparams['Legend']='Run 3 Unpacked Loose Iso. '
        col=ROOT.kRed
        pparams['MarkerColor']= col; pparams['LineColor'] = col;
        pparams['LineSize'] = 2
        pparams['Options']='l'
        pparams['isTH1']=False
        aplot = Plot(Name=hist.GetName(), Histo=hist,**pparams)
        aplot.normalize = False ;aplot.scaleFactor = None
        plots[-1].addPlot(aplot)

    #     i+=1
        tag='Tight Iso. Run3 Prelim'   
        print("\t",tag)
        hist=histsToPlot[tag][eT].Clone()
        hName=hist.GetName()
        pparams=getDefaultPlotParams(col=i,marker=3)
        pparams['Legend']="Tight Iso "
        c=ROOT.kMagenta
        pparams['MarkerColor']= c; pparams['LineColor'] = c;
        pparams['LineSize'] = 2
        pparams['Options']='l'
        pparams['isTH1']=False
        aplot = Plot(Name=hist.GetName(), Histo=hist,**pparams)
        aplot.normalize = False ;aplot.scaleFactor = None
#             aplot.normalize = True ;aplot.scaleFactor = 1.0*xSections[tag]
        plots[-1].addPlot(aplot)


        tag='Loose Iso. Run3 Prelim'   
        print("\t",tag)
        i+=1
        hist=histsToPlot[tag][eT].Clone()
        hName=hist.GetName()
        pparams=getDefaultPlotParams(col=i,marker=3)
        pparams['Legend']="Loose Iso. "
        c=ROOT.kGreen+2
        pparams['MarkerColor']= c; pparams['LineColor'] = c;
        pparams['LineSize'] = 2
        pparams['Options']='l'
        pparams['isTH1']=False
        aplot = Plot(Name=hist.GetName(), Histo=hist,**pparams)
        aplot.normalize = False ;aplot.scaleFactor = None
#             aplot.normalize = True ;aplot.scaleFactor = 1.0*xSections[tag]
        plots[-1].addPlot(aplot)

   
    canvas = []
    for plot in plots:
        canvas.append(plot.plot())             

#### Rates

In [ ]:
fileRate=ROOT.TFile('data/HistgramFile_Rate_Run3MC_2022v0_2_recalibV1_withPFA1.root')

Run3RateHist_SingleEG_rate_TightIso=fileRate.Get('SingleEG_rate_TightIso')
Run3RateHist_DoubleEG_rate_LooseIso=fileRate.Get('DoubleEG_rate_LooseIso')
Run3RateHist_SingleEG_rate_inclusive=fileRate.Get('SingleEG_rate_inclusive')

In [ ]:

col=[ 0,1,2,ROOT.kGreen+1,4,6,ROOT.kOrange+2,ROOT.kMagenta+2,9,10,11,12,13,14,15  ]
if True:  # Rates Isclusive / Isolated Tight / Isolated Loose
    plots=[]
    cPars=getCanvasParams('Run3MC')
    plot=getDefaultPlot(prefix=pltPrefix,name='Rates_prelimsVsUnpacked',cPars=cPars) 
    plot.legendPosition = (0.5,0.60,0.92,0.80)
    plot.descPosition   = (0.60,0.85)
    plot.desc =['#bf{Run III Prelims}','L1 EG']
    plot.yTitle = "Rate  [ kHz ]"  
    plot.xTitle = "E_{T}^{L1}"
    plot.xRange = (1.0,60.0) 
    plot.yRange = (1e-1,3e4) 
    plot.logy = True
    plots.append(plot)
    
    i=1
    
    i+=1
    
    
    
    hist=Run3RateHist_SingleEG_rate_inclusive.Clone()
    pparams=getDefaultPlotParams(col=i,marker=3)
    pparams['Legend']='Inclusive'
    pparams['MarkerColor']= col[i]; pparams['LineColor'] = col[i];
    pparams['MarkerStyle']= 22
    pparams['LineSize'] = 2
    pparams['Options']='pec'
    pparams['isTH1']=False
    aplot = Plot(Name=hist.GetName(), Histo=hist,**pparams)
    aplot.normalize = False ;aplot.scaleFactor = None
    plots[-1].addPlot(aplot)
    
    
    i+=1
    tag='Tight Iso. Run3 Prelim'
    hist=Run3RateHist_SingleEG_rate_TightIso.Clone()
    pparams=getDefaultPlotParams(col=i,marker=3)
    pparams['Legend']="Single EG , Tight Iso. online"
    pparams['MarkerColor']= col[i]; pparams['LineColor'] = col[i];
    pparams['MarkerStyle']= 22
    pparams['LineSize'] = 2
    pparams['Options']='pec'
    pparams['isTH1']=False
    aplot = Plot(Name=hist.GetName(), Histo=hist,**pparams)
    aplot.normalize = False ;aplot.scaleFactor = None
    plots[-1].addPlot(aplot)

    i+=1
    hist=Run3RateHist_DoubleEG_rate_LooseIso.Clone()
    pparams=getDefaultPlotParams(col=i,marker=3)
    pparams['Legend']="Double EG , Loose Iso. online"
    pparams['MarkerColor']= col[i]; pparams['LineColor'] = col[i];
    pparams['MarkerStyle']= 22
    pparams['LineSize'] = 2
    pparams['Options']='pec'
    pparams['isTH1']=False
    aplot = Plot(Name=hist.GetName(), Histo=hist,**pparams)
    aplot.normalize = False ;aplot.scaleFactor = None 
    plots[-1].addPlot(aplot)
    
    
    i+=1
    tag='Tight Iso. Run3 Prelim'
    hist=ratesToPlot[tag].Clone()
    pparams=getDefaultPlotParams(col=i,marker=3)
    pparams['Legend']="Single EG , Tight Iso."
    pparams['MarkerColor']= col[i]; pparams['LineColor'] = col[i];
    pparams['MarkerStyle']= 22
    pparams['LineSize'] = 2
    pparams['Options']='pec'
    pparams['isTH1']=False
    aplot = Plot(Name=hist.GetName(), Histo=hist,**pparams)
    aplot.normalize = False ;aplot.scaleFactor = None
    plots[-1].addPlot(aplot)

    i+=1
    tag='Loose Iso. Run3 Prelim'
    hist=ratesToPlot[tag].Clone()
    pparams=getDefaultPlotParams(col=i,marker=3)
    pparams['Legend']="Double EG , Loose Iso."
    pparams['MarkerColor']= col[i]; pparams['LineColor'] = col[i];
    pparams['MarkerStyle']= 22
    pparams['LineSize'] = 2
    pparams['Options']='pec'
    pparams['isTH1']=False
    aplot = Plot(Name=hist.GetName(), Histo=hist,**pparams)
    aplot.normalize = False ;aplot.scaleFactor = None 
    plots[-1].addPlot(aplot)
    canvas = []
    
    for plot in plots:
        canvas.append(plot.plot())            
 

## Final Plots

In [ ]:
fStore={}
fStore['pl32orig']=ROOT.TFile('workarea/gridV3_122X/gridA_v2/HistgramFile_step3step4_12XnewLayer1_Scan122XGridAp32.root')
fStore['pl32']=ROOT.TFile('workarea/gridV3_122X/gridA_v2/HistgramFile_step3step4_12XnewLayer1_Scan122XGridAp43.root')
fStore['pl16']=ROOT.TFile('workarea/gridV3_122X/gridA_v2/HistgramFile_step3step4_12XnewLayer1_Scan122XGridAp16.root')

hStore={}
hStore['pl32orig']=getTheObjectsFromFile(fStore['pl32orig'])
hStore['pl32']=getTheObjectsFromFile(fStore['pl32'])
hStore['pl16']=getTheObjectsFromFile(fStore['pl16'])

In [ ]:
histsToPlot={}
fTag='pl32orig'
opt='281'
eTs_toget=[15,18,20,22,25,28,30,32,36]
histsToPlot['Option '+opt]={}
for hName in hStore[fTag]['turn_on_progression'].keys():
    if opt not in hName:
        continue
    eT=int(hName.split('_')[8])
    if eT not in eTs_toget:
        continue
    print("Got ",hName)
    histsToPlot['Option '+opt][eT]=hStore[fTag]['turn_on_progression'][hName]
print()



fTag='pl32'
opt='387'
eTs_toget=[15,18,20,22,25,28,30,32,36]
histsToPlot['Option '+opt]={}
for hName in hStore[fTag]['turn_on_progression'].keys():
    if opt not in hName:
        continue
    eT=int(hName.split('_')[8])
    if eT not in eTs_toget:
        continue
    print("Got ",hName)
    histsToPlot['Option '+opt][eT]=hStore[fTag]['turn_on_progression'][hName]
print()

# histsToPlot['Tight Iso.' ]=histsToPlot.pop('Option '+opt)

fTag='pl16'
opt='137'
histsToPlot['Option '+opt]={}
for hName in hStore[fTag]['turn_on_progression'].keys():
    if opt not in hName:
        continue
    eT=int(hName.split('_')[8])
    if eT not in eTs_toget:
        continue
    print("Got ",hName)
    histsToPlot['Option '+opt][eT]=hStore[fTag]['turn_on_progression'][hName]
print()

# histsToPlot['Loose Iso.' ]=histsToPlot.pop('Option '+opt)

In [ ]:
histsToPlot['Option 387'][25]

In [ ]:
histsToPlot['Option 281'][25]

In [ ]:
histsToPlot['Option 137'][25].Integral()

In [ ]:
histsToPlot['Option 281'][25].Integral()

In [ ]:
fileRun3DefaultTurnOns=ROOT.TFile('data/run3MC_122X_Default_2022_v0_2_turnon.root')
run3DefaultTurnons=getTheObjectsFromFile(fileRun3DefaultTurnOns)

In [ ]:
Run3DefaultTurnOnMap={}
Run3DefaultTightTurnOnMap={}
Run3DefaultLooseTurnOnMap={}

for ky in  fileRun3DefaultTurnOns.GetListOfKeys():
    hName=ky.GetName()
    if 'Efficiency' not in hName:
        continue
    if 'tight' in hName:
        continue
    if 'loose' in hName:
        continue
    if 'def' in hName:
        continue
    if 'L1Et' not in hName:
        continue
    eT=int(hName.replace('hist_L1Et','').replace('Efficiency',''))
    Run3DefaultTurnOnMap[eT]=fileRun3DefaultTurnOns.Get(hName)
    if eT%10==0:
        print('For ',eT,' Adding ',hName)
               
for ky in  fileRun3DefaultTurnOns.GetListOfKeys():
    hName=ky.GetName()
    if 'Efficiency' not in hName:
        continue
    if 'tight' not in hName:
        continue
    if 'L1Et' not in hName:
        continue
    if 'def' in hName:
        continue
#     print(hName)
    eT=int(hName.replace('hist_L1Et','').replace('_tightisoEfficiency',''))
    Run3DefaultTightTurnOnMap[eT]=fileRun3DefaultTurnOns.Get(hName)
    if eT%10==0:
        print('For ',eT,' Adding ',hName)

for ky in  fileRun3DefaultTurnOns.GetListOfKeys():
    hName=ky.GetName()
    if 'Efficiency' not in hName:
        continue
    if 'loose' not in hName:
        continue
    if 'L1Et' not in hName:
        continue
    if 'def' in hName:
        continue
#     print(hName)
    eT=int(hName.replace('hist_L1Et','').replace('_looseisoEfficiency',''))
    Run3DefaultLooseTurnOnMap[eT]=fileRun3DefaultTurnOns.Get(hName)
    if eT%10==0:
        print('For ',eT,' Adding ',hName)

In [ ]:
if False:  #  Plots for diff eTs ,  Recaliberated Default Vs Recaliberated Tight New LUT
    plots=[]
    opt='387'
    for eT in [20,25,28,30,32]:
        cPars=getCanvasParams('Run3MC')
        plot=getDefaultPlot(prefix=pltPrefix,name='final_'+opt+'_E'+str(eT)+'_vsNewCalib_SEGTightEfffic',cPars=cPars) 
        plot.legendPosition = (0.45,0.80,0.94,0.87)
        plot.descPosition   = (0.61,0.93)
        plot.desc =['#bf{L1EG , Run III Prelims}']
        plot.yTitle = "#epsilon"  
        plot.xTitle = "E_{T}^{offline}"
        plot.xRange = (10.0,40.0) 
    #         plot.yRange = (1e-3,yMax[hName]*1e1) 
        plot.yRange = (0.0,1.2) 
        plot.logx = False
        plot.logy = False
        plots.append(plot)

        i=1
        print("Et : ",eT)
        hist=ROOT.TGraphAsymmErrors(Run2TightTurnOnMap[eT]).Clone()
        pparams=getDefaultPlotParams(col=i,marker=3)
        pparams['Legend']='Recaliberated v0_2'+" E_{T} > "+str(eT)
        pparams['MarkerColor']= col[i]; pparams['LineColor'] = col[i];
        pparams['LineSize'] = 2
        pparams['Options']='l'
        pparams['isTH1']=False
        aplot = Plot(Name=hist.GetName(), Histo=hist,**pparams)
        aplot.normalize = False ;aplot.scaleFactor = None
        plots[-1].addPlot(aplot)

    #     i+=1
        tag='Tight Iso. Run3 Prelim'   
        print("\t",tag)
        hist=histsToPlot['Option '+opt][eT].Clone()
        hName=hist.GetName()
        pparams=getDefaultPlotParams(col=i,marker=3)
        pparams['Legend']="Tight Iso Run 3 Prelims"
        c=4
        pparams['MarkerColor']= c; pparams['LineColor'] = c;
        pparams['LineSize'] = 2
        pparams['Options']='l'
        pparams['isTH1']=False
        aplot = Plot(Name=hist.GetName(), Histo=hist,**pparams)
        aplot.normalize = False ;aplot.scaleFactor = None
#             aplot.normalize = True ;aplot.scaleFactor = 1.0*xSections[tag]
        plots[-1].addPlot(aplot)


   
    canvas = []
    for plot in plots:
        canvas.append(plot.plot())             

In [ ]:
if False:  #  Plots for diff eTs , Recaliberated Default Vs Recaliberated Loose New LUT
    plots=[]
    opt='137'
    for eT in [20,25,28,30,32]:
        cPars=getCanvasParams('Run3MC')
        plot=getDefaultPlot(prefix=pltPrefix,name='final_'+opt+'_E'+str(eT)+'_vsNewCalib_DEGLooseEfffic',cPars=cPars) 
        plot.legendPosition = (0.45,0.80,0.94,0.87)
        plot.descPosition   = (0.61,0.93)
        plot.desc =['#bf{L1EG , Run III Prelims}']
        plot.yTitle = "#epsilon"  
        plot.xTitle = "E_{T}^{offline}"
        plot.xRange = (10.0,40.0) 
    #         plot.yRange = (1e-3,yMax[hName]*1e1) 
        plot.yRange = (0.0,1.2) 
        plot.logx = False
        plot.logy = False
        plots.append(plot)

        i=1
        print("Et : ",eT)
        hist=ROOT.TGraphAsymmErrors(Run2LooseTurnOnMap[eT]).Clone()
        pparams=getDefaultPlotParams(col=i,marker=3)
        pparams['Legend']='Recaliberated v0_2'+" E_{T} > "+str(eT)
        pparams['MarkerColor']= col[i]; pparams['LineColor'] = col[i];
        pparams['LineSize'] = 2
        pparams['Options']='l'
        pparams['isTH1']=False
        aplot = Plot(Name=hist.GetName(), Histo=hist,**pparams)
        aplot.normalize = False ;aplot.scaleFactor = None
        plots[-1].addPlot(aplot)

    #     i+=1
        tag='Loose Iso. Run3 Prelim'   
        print("\t",tag)
        hist=histsToPlot['Option '+opt][eT].Clone()
        hName=hist.GetName()
        pparams=getDefaultPlotParams(col=i,marker=3)
        pparams['Legend']="Loose Iso Run 3 Prelims"
        c=4
        pparams['MarkerColor']= c; pparams['LineColor'] = c;
        pparams['LineSize'] = 2
        pparams['Options']='l'
        pparams['isTH1']=False
        aplot = Plot(Name=hist.GetName(), Histo=hist,**pparams)
        aplot.normalize = False ;aplot.scaleFactor = None
#             aplot.normalize = True ;aplot.scaleFactor = 1.0*xSections[tag]
        plots[-1].addPlot(aplot)


   
    canvas = []
    for plot in plots:
        canvas.append(plot.plot())             

In [ ]:
if False:  #  Plots for diff eTs ,  Default Vs Recaliberated Tight New LUT
    plots=[]
    opt='387'
    for eT in [20,25,28,30,32]:
        cPars=getCanvasParams('Run3MC')
        plot=getDefaultPlot(prefix=pltPrefix,name='final_'+opt+'_E'+str(eT)+'_vsOldCalib_SEGTightEfffic',cPars=cPars) 
        plot.legendPosition = (0.45,0.80,0.94,0.87)
        plot.descPosition   = (0.61,0.93)
        plot.desc =['#bf{L1EG , Run III Prelims}']
        plot.yTitle = "#epsilon"  
        plot.xTitle = "E_{T}^{offline}"
        plot.xRange = (10.0,50.0) 
    #         plot.yRange = (1e-3,yMax[hName]*1e1) 
        plot.yRange = (0.0,1.2) 
        plot.logx = False
        plot.logy = False
        plots.append(plot)

        i=1
        print("Et : ",eT)
        hist=ROOT.TGraphAsymmErrors(Run3DefaultTightTurnOnMap[eT]).Clone()
        pparams=getDefaultPlotParams(col=i,marker=3)
        pparams['Legend']='Default v0_2'+" E_{T} > "+str(eT)
        pparams['MarkerColor']= col[i]; pparams['LineColor'] = col[i];
        pparams['LineSize'] = 2
        pparams['Options']='l'
        pparams['isTH1']=False
        aplot = Plot(Name=hist.GetName(), Histo=hist,**pparams)
        aplot.normalize = False ;aplot.scaleFactor = None
        plots[-1].addPlot(aplot)

    #     i+=1
        tag='Tight Iso. Run3 Prelim'   
        print("\t",tag)
        hist=histsToPlot['Option '+opt][eT].Clone()
        hName=hist.GetName()
        pparams=getDefaultPlotParams(col=i,marker=3)
        pparams['Legend']="Tight Iso Run 3 Prelims"
        c=4
        pparams['MarkerColor']= c; pparams['LineColor'] = c;
        pparams['LineSize'] = 2
        pparams['Options']='l'
        pparams['isTH1']=False
        aplot = Plot(Name=hist.GetName(), Histo=hist,**pparams)
        aplot.normalize = False ;aplot.scaleFactor = None
#             aplot.normalize = True ;aplot.scaleFactor = 1.0*xSections[tag]
        plots[-1].addPlot(aplot)


   
    canvas = []
    for plot in plots:
        canvas.append(plot.plot())             

In [ ]:
if False:  #  Plots for diff eTs ,  Default Vs Recaliberated Loose New LUT
    plots=[]
    opt='137'
    for eT in [20,25,28,30,32]:
        cPars=getCanvasParams('Run3MC')
        plot=getDefaultPlot(prefix=pltPrefix,name='final_'+opt+'_E'+str(eT)+'_vsOldCalib_DEGLooseEfffic',cPars=cPars) 
        plot.legendPosition = (0.45,0.80,0.94,0.87)
        plot.descPosition   = (0.61,0.93)
        plot.desc =['#bf{L1EG , Run III Prelims}']
        plot.yTitle = "#epsilon"  
        plot.xTitle = "E_{T}^{offline}"
        plot.xRange = (10.0,40.0) 
    #         plot.yRange = (1e-3,yMax[hName]*1e1) 
        plot.yRange = (0.0,1.2) 
        plot.logx = False
        plot.logy = False
        plots.append(plot)

        i=1
        print("Et : ",eT)
        hist=ROOT.TGraphAsymmErrors(Run3DefaultLooseTurnOnMap[eT]).Clone()
        pparams=getDefaultPlotParams(col=i,marker=3)
        pparams['Legend']='Default v0_2'+" E_{T} > "+str(eT)
        pparams['MarkerColor']= col[i]; pparams['LineColor'] = col[i];
        pparams['LineSize'] = 2
        pparams['Options']='l'
        pparams['isTH1']=False
        aplot = Plot(Name=hist.GetName(), Histo=hist,**pparams)
        aplot.normalize = False ;aplot.scaleFactor = None
        plots[-1].addPlot(aplot)

    #     i+=1
        tag='Loose Iso. Run3 Prelim'   
        print("\t",tag)
        hist=histsToPlot['Option '+opt][eT].Clone()
        hName=hist.GetName()
        pparams=getDefaultPlotParams(col=i,marker=3)
        pparams['Legend']="Loose Iso Run 3 Prelims"
        c=4
        pparams['MarkerColor']= c; pparams['LineColor'] = c;
        pparams['LineSize'] = 2
        pparams['Options']='l'
        pparams['isTH1']=False
        aplot = Plot(Name=hist.GetName(), Histo=hist,**pparams)
        aplot.normalize = False ;aplot.scaleFactor = None
#             aplot.normalize = True ;aplot.scaleFactor = 1.0*xSections[tag]
        plots[-1].addPlot(aplot)


   
    canvas = []
    for plot in plots:
        canvas.append(plot.plot())             

### Final Rates

In [ ]:
ratesToPlotV2={}
fTag='pl32orig'
opt='281'
for hName in hStore[fTag]['rate_progression'].keys():
    if opt not in hName:
        continue
    print("Got ",hName)
    ratesToPlotV2['Option '+opt]=hStore[fTag]['rate_progression'][hName]
print()

fTag='pl16'
opt='137'
ratesToPlotV2['Option '+opt]={}
for hName in hStore[fTag]['rate_progression'].keys():
    if ''+opt+'_' not in hName:
        continue
    print("Got ",hName)
    ratesToPlotV2['Option '+opt]=hStore[fTag]['rate_progression'][hName]

fTag='pl32'
opt='387'
for hName in hStore[fTag]['rate_progression'].keys():
    if opt not in hName:
        continue
    print("Got ",hName)
    ratesToPlotV2['Option '+opt]=hStore[fTag]['rate_progression'][hName]
print()

ratesToPlotV2['Tight Iso. Run3 Prelim' ]=ratesToPlotV2.pop('Option '+opt)

fTag='pl16'
opt='137'
ratesToPlotV2['DOption '+opt]={}
for hName in hStore[fTag]['rate_progression_double'].keys():
    if ''+opt+'_' not in hName:
        continue
    print("Got ",hName)
    ratesToPlotV2['DOption '+opt]=hStore[fTag]['rate_progression_double'][hName]
    
print()
ratesToPlotV2['Loose Iso. Run3 Prelim' ]=ratesToPlotV2.pop('DOption '+opt)

In [ ]:
ratesToPlotV2

In [ ]:
if False:  # Rates Isclusive / Isolated Tight / Isolated Loose
    plots=[]
    cPars=getCanvasParams('Run3MC')
    plot=getDefaultPlot(prefix=pltPrefix,name='Rates_Final_',cPars=cPars) 
    plot.legendPosition = (0.45,0.58,0.92,0.70)
    plot.descPosition   = (0.60,0.80)
    plot.desc =['#bf{Run III Prelims}','L1 EG']
    plot.yTitle = "Rate  [ kHz ]"  
    plot.xTitle = "E_{T}^{L1}"
    plot.xRange = (1.0,60.0) 
    plot.yRange = (1e-1,3e4) 
    plot.logy = True
    plots.append(plot)
    
    i=1
    
    i+=1
    hist=Run2RateHist_SingleEG_rate_inclusive.Clone()
    pparams=getDefaultPlotParams(col=i,marker=3)
    pparams['Legend']='Inclusive'
    pparams['MarkerColor']= col[i]; pparams['LineColor'] = col[i];
    pparams['MarkerStyle']= 22
    pparams['LineSize'] = 2
    pparams['Options']='pec'
    pparams['isTH1']=False
    aplot = Plot(Name=hist.GetName(), Histo=hist,**pparams)
    aplot.normalize = False ;aplot.scaleFactor = None
    plots[-1].addPlot(aplot)
    
    
    i+=1
    tag='Tight Iso. Run3 Prelim'
    hist=ratesToPlotV2[tag].Clone()
    pparams=getDefaultPlotParams(col=i,marker=3)
    pparams['Legend']="Single EG , Tight Iso. [Opt :287]"
    pparams['MarkerColor']= col[i]; pparams['LineColor'] = col[i];
    pparams['MarkerStyle']= 22
    pparams['LineSize'] = 2
    pparams['Options']='pec'
    pparams['isTH1']=False
    aplot = Plot(Name=hist.GetName(), Histo=hist,**pparams)
    aplot.normalize = False ;aplot.scaleFactor = None
    plots[-1].addPlot(aplot)

    i+=1
    tag='Loose Iso. Run3 Prelim'
    hist=ratesToPlotV2[tag].Clone()
    pparams=getDefaultPlotParams(col=i,marker=3)
    pparams['Legend']="Double EG , Loose Iso. [Opt :137]"
    pparams['MarkerColor']= col[i]; pparams['LineColor'] = col[i];
    pparams['MarkerStyle']= 22
    pparams['LineSize'] = 2
    pparams['Options']='pec'
    pparams['isTH1']=False
    aplot = Plot(Name=hist.GetName(), Histo=hist,**pparams)
    aplot.normalize = False ;aplot.scaleFactor = None 
    plots[-1].addPlot(aplot)
    canvas = []
    
    for plot in plots:
        canvas.append(plot.plot())            
 

#### TIGHT COMPARISON

In [ ]:
if False:  # Rates Tight Iso : Comparison vs Old
    plots=[]
    cPars=getCanvasParams('Run3MC')
    plot=getDefaultPlot(prefix=pltPrefix,name='Rates_Final_tightIsoVsOld',cPars=cPars) 
    plot.legendPosition = (0.45,0.58,0.92,0.70)
    plot.descPosition   = (0.60,0.80)
    plot.desc =['#bf{Run III Prelims}','L1 EG']
    plot.yTitle = "Rate  [ kHz ]"  
    plot.xTitle = "E_{T}^{L1}"
    plot.xRange = (1.0,60.0) 
    plot.yRange = (1e-1,3e4) 
    plot.logy = True
    plots.append(plot)
    
    i=1
    
    i+=1
    hist=Run3DeafultRateHist_SingleEG_rate_TightIso.Clone()
    pparams=getDefaultPlotParams(col=i,marker=3)
    pparams['Legend']='Default v0_2'
    pparams['MarkerColor']= col[i]; pparams['LineColor'] = col[i];
    pparams['MarkerStyle']= 22
    pparams['LineSize'] = 2
    pparams['Options']='pec'
    pparams['isTH1']=False
    aplot = Plot(Name=hist.GetName(), Histo=hist,**pparams)
    aplot.normalize = False ;aplot.scaleFactor = None
    plots[-1].addPlot(aplot)
    
    i+=1
    tag='Tight Iso. Run3 Prelim'
    hist=ratesToPlotV2[tag].Clone()
    pparams=getDefaultPlotParams(col=i,marker=3)
    pparams['Legend']="Single EG , Tight Iso. [Opt :287]"
    pparams['MarkerColor']= col[i]; pparams['LineColor'] = col[i];
    pparams['MarkerStyle']= 22
    pparams['LineSize'] = 2
    pparams['Options']='pec'
    pparams['isTH1']=False
    aplot = Plot(Name=hist.GetName(), Histo=hist,**pparams)
    aplot.normalize = False ;aplot.scaleFactor = None
    plots[-1].addPlot(aplot)


    for plot in plots:
        canvas.append(plot.plot())            
        
if False:  # Rates Tight Iso : Comparison vs Recaliberated
    plots=[]
    cPars=getCanvasParams('Run3MC')
    plot=getDefaultPlot(prefix=pltPrefix,name='Rates_Final_tightIsoVsNew',cPars=cPars) 
    plot.legendPosition = (0.45,0.58,0.92,0.70)
    plot.descPosition   = (0.60,0.80)
    plot.desc =['#bf{Run III Prelims}','L1 EG']
    plot.yTitle = "Rate  [ kHz ]"  
    plot.xTitle = "E_{T}^{L1}"
    plot.xRange = (1.0,60.0) 
    plot.yRange = (1e-1,3e4) 
    plot.logy = True
    plots.append(plot)
    
    i=1
    
    i+=1
    hist=Run2RateHist_SingleEG_rate_TightIso.Clone()
    pparams=getDefaultPlotParams(col=i,marker=3)
    pparams['Legend']='Recaliberated v0_2'
    pparams['MarkerColor']= col[i]; pparams['LineColor'] = col[i];
    pparams['MarkerStyle']= 22
    pparams['LineSize'] = 2
    pparams['Options']='pec'
    pparams['isTH1']=False
    aplot = Plot(Name=hist.GetName(), Histo=hist,**pparams)
    aplot.normalize = False ;aplot.scaleFactor = None
    plots[-1].addPlot(aplot)
    
    i+=1
    tag='Tight Iso. Run3 Prelim'
    hist=ratesToPlotV2[tag].Clone()
    pparams=getDefaultPlotParams(col=i,marker=3)
    pparams['Legend']="Single EG , Tight Iso. [Opt :287]"
    pparams['MarkerColor']= col[i]; pparams['LineColor'] = col[i];
    pparams['MarkerStyle']= 22
    pparams['LineSize'] = 2
    pparams['Options']='pec'
    pparams['isTH1']=False
    aplot = Plot(Name=hist.GetName(), Histo=hist,**pparams)
    aplot.normalize = False ;aplot.scaleFactor = None
    plots[-1].addPlot(aplot)


    for plot in plots:
        canvas.append(plot.plot())            
         

### Loose Comparison

In [ ]:
if False:  # Rates Loose Iso : Comparison vs Old
    plots=[]
    cPars=getCanvasParams('Run3MC')
    plot=getDefaultPlot(prefix=pltPrefix,name='Rates_Final_looseIsoVsOld',cPars=cPars) 
    plot.legendPosition = (0.45,0.58,0.92,0.70)
    plot.descPosition   = (0.60,0.80)
    plot.desc =['#bf{Run III Prelims}','L1 EG']
    plot.yTitle = "Rate  [ kHz ]"  
    plot.xTitle = "E_{T}^{L1}"
    plot.xRange = (1.0,60.0) 
    plot.yRange = (1e-1,3e4) 
    plot.logy = True
    plots.append(plot)
    
    i=1
    
    i+=1
    hist=Run3DeafultRateHist_DoubleEG_rate_LooseIso.Clone()
    pparams=getDefaultPlotParams(col=i,marker=3)
    pparams['Legend']='Default v0_2'
    pparams['MarkerColor']= col[i]; pparams['LineColor'] = col[i];
    pparams['MarkerStyle']= 22
    pparams['LineSize'] = 2
    pparams['Options']='pec'
    pparams['isTH1']=False
    aplot = Plot(Name=hist.GetName(), Histo=hist,**pparams)
    aplot.normalize = False ;aplot.scaleFactor = None
    plots[-1].addPlot(aplot)
    
    i+=1
    tag='Loose Iso. Run3 Prelim'
    hist=ratesToPlotV2[tag].Clone()
    pparams=getDefaultPlotParams(col=i,marker=3)
    pparams['Legend']="Double EG , Loose Iso. [Opt :137]"
    pparams['MarkerColor']= col[i]; pparams['LineColor'] = col[i];
    pparams['MarkerStyle']= 22
    pparams['LineSize'] = 2
    pparams['Options']='pec'
    pparams['isTH1']=False
    aplot = Plot(Name=hist.GetName(), Histo=hist,**pparams)
    aplot.normalize = False ;aplot.scaleFactor = None
    plots[-1].addPlot(aplot)


    for plot in plots:
        canvas.append(plot.plot())            
        
if False:  # Rates Loose Iso : Comparison vs Recaliberated
    plots=[]
    cPars=getCanvasParams('Run3MC')
    plot=getDefaultPlot(prefix=pltPrefix,name='Rates_Final_looseIsoVsNew',cPars=cPars) 
    plot.legendPosition = (0.45,0.58,0.92,0.70)
    plot.descPosition   = (0.60,0.80)
    plot.desc =['#bf{Run III Prelims}','L1 EG']
    plot.yTitle = "Rate  [ kHz ]"  
    plot.xTitle = "E_{T}^{L1}"
    plot.xRange = (1.0,60.0) 
    plot.yRange = (1e-1,3e4) 
    plot.logy = True
    plots.append(plot)
    
    i=1
    
    i+=1
    hist=Run2RateHist_DoubleEG_rate_LooseIso.Clone()
    pparams=getDefaultPlotParams(col=i,marker=3)
    pparams['Legend']='Recaliberated v0_2'
    pparams['MarkerColor']= col[i]; pparams['LineColor'] = col[i];
    pparams['MarkerStyle']= 22
    pparams['LineSize'] = 2
    pparams['Options']='pec'
    pparams['isTH1']=False
    aplot = Plot(Name=hist.GetName(), Histo=hist,**pparams)
    aplot.normalize = False ;aplot.scaleFactor = None
    plots[-1].addPlot(aplot)
    
    i+=1
    tag='Loose Iso. Run3 Prelim'
    hist=ratesToPlotV2[tag].Clone()
    pparams=getDefaultPlotParams(col=i,marker=3)
    pparams['Legend']="Double EG , Loose Iso. [Opt :137]"
    pparams['MarkerColor']= col[i]; pparams['LineColor'] = col[i];
    pparams['MarkerStyle']= 22
    pparams['LineSize'] = 2
    pparams['Options']='pec'
    pparams['isTH1']=False
    aplot = Plot(Name=hist.GetName(), Histo=hist,**pparams)
    aplot.normalize = False ;aplot.scaleFactor = None
    plots[-1].addPlot(aplot)


    for plot in plots:
        canvas.append(plot.plot())            
         

In [ ]:
if False:  # Rates Inclusive Single EG Comparison vs Old
    plots=[]
    cPars=getCanvasParams('Run3MC')
    plot=getDefaultPlot(prefix=pltPrefix,name='Rates_Final_ijnclusiveNewVsOld',cPars=cPars) 
    plot.legendPosition = (0.45,0.58,0.92,0.70)
    plot.descPosition   = (0.60,0.80)
    plot.desc =['#bf{Run III Prelims}','L1 EG']
    plot.yTitle = "Rate  [ kHz ]"  
    plot.xTitle = "E_{T}^{L1}"
    plot.xRange = (1.0,60.0) 
    plot.yRange = (1e-1,3e4) 
    plot.logy = True
    plots.append(plot)
    
    i=1
    
    i+=1
    hist=Run3DeafultRateHist_SingleEG_rate_inclusive.Clone()
    pparams=getDefaultPlotParams(col=i,marker=3)
    pparams['Legend']='Default v0_2'
    pparams['MarkerColor']= col[i]; pparams['LineColor'] = col[i];
    pparams['MarkerStyle']= 22
    pparams['LineSize'] = 2
    pparams['Options']='pec'
    pparams['isTH1']=False
    aplot = Plot(Name=hist.GetName(), Histo=hist,**pparams)
    aplot.normalize = False ;aplot.scaleFactor = None
    plots[-1].addPlot(aplot)
    
    i+=1
    tag='Loose Iso. Run3 Prelim'
    hist=Run2RateHist_SingleEG_rate_inclusive.Clone()
    pparams=getDefaultPlotParams(col=i,marker=3)
    pparams['Legend']='Recaliberated v0_2'
    pparams['MarkerColor']= col[i]; pparams['LineColor'] = col[i];
    pparams['MarkerStyle']= 22
    pparams['LineSize'] = 2
    pparams['Options']='pec'
    pparams['isTH1']=False
    aplot = Plot(Name=hist.GetName(), Histo=hist,**pparams)
    aplot.normalize = False ;aplot.scaleFactor = None
    plots[-1].addPlot(aplot)


    for plot in plots:
        canvas.append(plot.plot())            

## Efficiency Plots

In [ ]:
histsToPlot

In [ ]:
histsToPlot['Option 137'][eT].Integral()

In [ ]:
if False:  #  Plots for diff eTs , inclusive / tight and loose in same one
    plots=[]
    for eT in eTs_toget:
        cPars=getCanvasParams('Run3MC')
#         plot=getDefaultPlot(prefix=pltPrefix,name='macroscopic/Final_E'+str(eT)+'_AllEffficiencies',cPars=cPars) 
#         plot.legendPosition = (0.65-0.4,0.57,0.94-0.4,0.75)
        plot=getDefaultPlot(prefix=pltPrefix,name='Final_E'+str(eT)+'_AllEffficiencies',cPars=cPars) 
        plot.legendPosition = (0.65,0.57,0.94,0.75)
        plot.descPosition   = (0.65,0.80)
        plot.desc =['#bf{Run III Prelims}','L1 EG']
        plot.yTitle = "#epsilon"  
        plot.xTitle = "E_{T}^{offline}"
        plot.xRange = (10.0,100.0) 
    #         plot.yRange = (1e-3,yMax[hName]*1e1) 
        plot.yRange = (0.0,1.02) 
        plot.logx = True
        plot.logy = False
        plots.append(plot)

        i=1
        print("Et : ",eT)
        hist=ROOT.TGraphAsymmErrors(Run2TurnOnMap[eT]).Clone()
        pparams=getDefaultPlotParams(col=i,marker=3)
        pparams['Legend']='Inclusive'+" E_{T} > "+str(eT)
        pparams['MarkerColor']= col[i]; pparams['LineColor'] = col[i];
        pparams['LineSize'] = 2
        pparams['Options']='l'
        pparams['isTH1']=False
        aplot = Plot(Name=hist.GetName(), Histo=hist,**pparams)
        aplot.normalize = False ;aplot.scaleFactor = None
        plots[-1].addPlot(aplot)

    #     i+=1
        tag='Tight Iso. Run3 Prelim'   
        print("\t",tag)
        hist=histsToPlot['Option 387'][eT].Clone()
        hName=hist.GetName()
        pparams=getDefaultPlotParams(col=i,marker=3)
        pparams['Legend']="Tight Iso ,"+" E_{T} > "+str(eT)
        c=4
        pparams['MarkerColor']= c; pparams['LineColor'] = c;
        pparams['LineSize'] = 2
        pparams['Options']='l'
        pparams['isTH1']=False
        aplot = Plot(Name=hist.GetName(), Histo=hist,**pparams)
        aplot.normalize = False ;aplot.scaleFactor = None
#             aplot.normalize = True ;aplot.scaleFactor = 1.0*xSections[tag]
        plots[-1].addPlot(aplot)


        tag='Loose Iso. Run3 Prelim'   
        print("\t",tag)
        i+=1
        hist=histsToPlot['Option 137'][eT].Clone()
        hName=hist.GetName()
        pparams=getDefaultPlotParams(col=i,marker=3)
        pparams['Legend']="Loose Iso. "+", E_{T} > "+str(eT)
        c=2
        pparams['MarkerColor']= c; pparams['LineColor'] = c;
        pparams['LineSize'] = 2
        pparams['Options']='l'
        pparams['isTH1']=False
        aplot = Plot(Name=hist.GetName(), Histo=hist,**pparams)
        aplot.normalize = False ;aplot.scaleFactor = None
#             aplot.normalize = True ;aplot.scaleFactor = 1.0*xSections[tag]
        plots[-1].addPlot(aplot)

   
    canvas = []
    for plot in plots:
        canvas.append(plot.plot())             

In [ ]:
histsToPlot['Tight Iso. Run3 Prelim'].keys()

### CHECKECS

In [ ]:
if True:  # MICROSCOPIC  Plots for diff eTs , inclusive / tight and loose in same one
    plots=[]
    for eT in [20, 22, 25, 28, 30, 32, 34]:
        cPars=getCanvasParams('Run3MC')
#         plot=getDefaultPlot(prefix=pltPrefix,name='macroscopic/Final_E'+str(eT)+'_AllEffficiencies',cPars=cPars) 
#         plot.legendPosition = (0.65-0.4,0.57,0.94-0.4,0.75)
        plot=getDefaultPlot(prefix=pltPrefix,name='macroscopic/Final_E'+str(eT)+'_AllEffficiencies',cPars=cPars) 
        plot.legendPosition = (0.65,0.2,0.94,0.42)
        plot.descPosition   = (0.65,0.60)
        plot.desc =['#bf{Run III Prelims}','L1 EG']
        plot.yTitle = "#epsilon"  
        plot.xTitle = "E_{T}^{offline}"
        plot.xRange = (20.0,50.0) 
    #         plot.yRange = (1e-3,yMax[hName]*1e1) 
        plot.yRange = (0.8,1.02) 
        plot.logx = False
        plot.logy = False
        plots.append(plot)

        i=1
        print("Et : ",eT)
        hist=ROOT.TGraphAsymmErrors(Run2TurnOnMap[eT]).Clone()
        pparams=getDefaultPlotParams(col=i,marker=3)
        pparams['Legend']='Inclusive'+" E_{T} > "+str(eT)
        pparams['MarkerColor']= col[i]; pparams['LineColor'] = col[i];
        pparams['LineSize'] = 2
        pparams['Options']='l'
        pparams['isTH1']=False
        aplot = Plot(Name=hist.GetName(), Histo=hist,**pparams)
        aplot.normalize = False ;aplot.scaleFactor = None
        plots[-1].addPlot(aplot)

    #     i+=1
        tag='Tight Iso. Run3 Prelim'   
        print("\t",tag)
        hist=histsToPlot[tag][eT].Clone()
        hName=hist.GetName()
        pparams=getDefaultPlotParams(col=i,marker=3)
        pparams['Legend']="Tight Iso ,"+" E_{T} > "+str(eT)
        c=4
        pparams['MarkerColor']= c; pparams['LineColor'] = c;
        pparams['LineSize'] = 2
        pparams['Options']='l'
        pparams['isTH1']=False
        aplot = Plot(Name=hist.GetName(), Histo=hist,**pparams)
        aplot.normalize = False ;aplot.scaleFactor = None
#             aplot.normalize = True ;aplot.scaleFactor = 1.0*xSections[tag]
        plots[-1].addPlot(aplot)


        tag='Loose Iso. Run3 Prelim'   
        print("\t",tag)
        i+=1
        hist=histsToPlot[tag][eT].Clone()
        hName=hist.GetName()
        pparams=getDefaultPlotParams(col=i,marker=3)
        pparams['Legend']="Loose Iso. "+", E_{T} > "+str(eT)
        c=2
        pparams['MarkerColor']= c; pparams['LineColor'] = c;
        pparams['LineSize'] = 2
        pparams['Options']='l'
        pparams['isTH1']=False
        aplot = Plot(Name=hist.GetName(), Histo=hist,**pparams)
        aplot.normalize = False ;aplot.scaleFactor = None
#             aplot.normalize = True ;aplot.scaleFactor = 1.0*xSections[tag]
        plots[-1].addPlot(aplot)

   
    canvas = []
    for plot in plots:
        canvas.append(plot.plot())             

In [ ]:
ratesToPlotV2

In [ ]:
if True:  # Rates Tight Iso : Comparison vs Recaliberated
    plots=[]
    cPars=getCanvasParams('Run3MC')
    plot=getDefaultPlot(prefix=pltPrefix,name='Rates_DCHK_tightIsoVsNew',cPars=cPars) 
    plot.legendPosition = (0.45,0.58,0.92,0.70)
    plot.descPosition   = (0.60,0.80)
    plot.desc =['#bf{Run III Prelims}','L1 EG']
    plot.yTitle = "Rate  [ kHz ]"  
    plot.xTitle = "E_{T}^{L1}"
    plot.xRange = (1.0,60.0) 
    plot.yRange = (1e-1,3e4) 
    plot.logy = True
    plots.append(plot)
    
    i=1
    
    i+=1
    hist=Run2RateHist_SingleEG_rate_TightIso.Clone()
    pparams=getDefaultPlotParams(col=i,marker=3)
    pparams['Legend']='Recaliberated v0_2'
    pparams['MarkerColor']= col[i]; pparams['LineColor'] = col[i];
    pparams['MarkerStyle']= 22
    pparams['LineSize'] = 2
    pparams['Options']='pec'
    pparams['isTH1']=False
    aplot = Plot(Name=hist.GetName(), Histo=hist,**pparams)
    aplot.normalize = False ;aplot.scaleFactor = None
    plots[-1].addPlot(aplot)
       
   
    i+=1
    tag='Option 137'
    hist=ratesToPlotV2[tag].Clone()
    pparams=getDefaultPlotParams(col=i,marker=3)
    pparams['Legend']=tag
    pparams['MarkerColor']= col[i]; pparams['LineColor'] = col[i];
    pparams['MarkerStyle']= 22
    pparams['LineSize'] = 2
    pparams['Options']='pec'
    pparams['isTH1']=False
    aplot = Plot(Name=hist.GetName(), Histo=hist,**pparams)
    aplot.normalize = False ;aplot.scaleFactor = None
    plots[-1].addPlot(aplot)
   
    i+=1
    tag='Option 281'
    hist=ratesToPlotV2[tag].Clone()
    pparams=getDefaultPlotParams(col=i,marker=3)
    pparams['Legend']=tag
    pparams['MarkerColor']= col[i]; pparams['LineColor'] = col[i];
    pparams['MarkerStyle']= 22
    pparams['LineSize'] = 2
    pparams['Options']='pec'
    pparams['isTH1']=False
    aplot = Plot(Name=hist.GetName(), Histo=hist,**pparams)
    aplot.normalize = False ;aplot.scaleFactor = None
    plots[-1].addPlot(aplot)


    for plot in plots:
        canvas.append(plot.plot())            
         

In [ ]:
if False:  #  Plots for diff eTs ,  Recaliberated Default Vs Recaliberated Tight New LUT
    plots=[]
    opt='387'
    for eT in [20,25,28,30,32]:
        cPars=getCanvasParams('Run3MC')
        plot=getDefaultPlot(prefix=pltPrefix,name='387vs281_'+opt+'_E'+str(eT)+'_vsNewCalib_SEGTightEfffic',cPars=cPars) 
        plot.legendPosition = (0.45,0.80,0.94,0.87)
        plot.descPosition   = (0.61,0.93)
        plot.desc =['#bf{L1EG , Run III Prelims}']
        plot.yTitle = "#epsilon"  
        plot.xTitle = "E_{T}^{offline}"
        plot.xRange = (10.0,40.0) 
    #         plot.yRange = (1e-3,yMax[hName]*1e1) 
        plot.yRange = (0.0,1.2) 
        plot.logx = False
        plot.logy = False
        plots.append(plot)

        i=1
        print("Et : ",eT)
        hist=ROOT.TGraphAsymmErrors(Run2TightTurnOnMap[eT]).Clone()
        pparams=getDefaultPlotParams(col=i,marker=3)
        pparams['Legend']='Recaliberated v0_2'+" E_{T} > "+str(eT)
        pparams['MarkerColor']= col[i]; pparams['LineColor'] = col[i];
        pparams['LineSize'] = 2
        pparams['Options']='l'
        pparams['isTH1']=False
        aplot = Plot(Name=hist.GetName(), Histo=hist,**pparams)
        aplot.normalize = False ;aplot.scaleFactor = None
        plots[-1].addPlot(aplot)

    #     i+=1
        tag='Tight Iso. Run3 Prelim'   
        print("\t",tag)
        hist=histsToPlot['Option '+opt][eT].Clone()
        hName=hist.GetName()
        pparams=getDefaultPlotParams(col=i,marker=3)
        pparams['Legend']="387"
        c=4
        pparams['MarkerColor']= c; pparams['LineColor'] = c;
        pparams['LineSize'] = 2
        pparams['Options']='l'
        pparams['isTH1']=False
        aplot = Plot(Name=hist.GetName(), Histo=hist,**pparams)
        aplot.normalize = False ;aplot.scaleFactor = None
#             aplot.normalize = True ;aplot.scaleFactor = 1.0*xSections[tag]
        plots[-1].addPlot(aplot)

        tag='Tight Iso. Run3 Prelim'   
        print("\t",tag)
        opt='281'
        hist=histsToPlot['Option '+opt][eT].Clone()
        hName=hist.GetName()
        pparams=getDefaultPlotParams(col=i,marker=3)
        pparams['Legend']="281"
        c=2
        pparams['MarkerColor']= c; pparams['LineColor'] = c;
        pparams['LineSize'] = 2
        pparams['Options']='l'
        pparams['isTH1']=False
        aplot = Plot(Name=hist.GetName(), Histo=hist,**pparams)
        aplot.normalize = False ;aplot.scaleFactor = None
#             aplot.normalize = True ;aplot.scaleFactor = 1.0*xSections[tag]
        plots[-1].addPlot(aplot)

    
   
    canvas = []
    for plot in plots:
        canvas.append(plot.plot())             
        
if False:  #  Macroscopi Plots for diff eTs ,  Recaliberated Default Vs Recaliberated Tight New LUT
    plots=[]
    opt='387'
    for eT in [20,25,28,30,32]:
        cPars=getCanvasParams('Run3MC')
        plot=getDefaultPlot(prefix=pltPrefix,name='macroscopic/387vs281_'+opt+'_E'+str(eT)+'_vsNewCalib_SEGTightEfffic',cPars=cPars) 
        plot.legendPosition = (0.65,0.2,0.94,0.42)
        plot.descPosition   = (0.65,0.60)
        plot.desc =['#bf{Run III Prelims}','L1 EG']
        plot.yTitle = "#epsilon"  
        plot.xTitle = "E_{T}^{offline}"
        plot.xRange = (15.0,40.0) 
    #         plot.yRange = (1e-3,yMax[hName]*1e1) 
        plot.yRange = (0.8,1.02) 
        plot.logx = False
        plot.logy = False
        plots.append(plot)

        i=1
        print("Et : ",eT)
        hist=ROOT.TGraphAsymmErrors(Run2TightTurnOnMap[eT]).Clone()
        pparams=getDefaultPlotParams(col=i,marker=3)
        pparams['Legend']='Recaliberated v0_2'+" E_{T} > "+str(eT)
        pparams['MarkerColor']= col[i]; pparams['LineColor'] = col[i];
        pparams['LineSize'] = 2
        pparams['Options']='l'
        pparams['isTH1']=False
        aplot = Plot(Name=hist.GetName(), Histo=hist,**pparams)
        aplot.normalize = False ;aplot.scaleFactor = None
        plots[-1].addPlot(aplot)

    #     i+=1
        tag='Tight Iso. Run3 Prelim'   
        print("\t",tag)
        hist=histsToPlot['Option '+opt][eT].Clone()
        hName=hist.GetName()
        pparams=getDefaultPlotParams(col=i,marker=3)
        pparams['Legend']="387"
        c=4
        pparams['MarkerColor']= c; pparams['LineColor'] = c;
        pparams['LineSize'] = 2
        pparams['Options']='l'
        pparams['isTH1']=False
        aplot = Plot(Name=hist.GetName(), Histo=hist,**pparams)
        aplot.normalize = False ;aplot.scaleFactor = None
#             aplot.normalize = True ;aplot.scaleFactor = 1.0*xSections[tag]
        plots[-1].addPlot(aplot)

        tag='Tight Iso. Run3 Prelim'   
        print("\t",tag)
        opt='281'
        hist=histsToPlot['Option '+opt][eT].Clone()
        hName=hist.GetName()
        pparams=getDefaultPlotParams(col=i,marker=3)
        pparams['Legend']="281"
        c=2
        pparams['MarkerColor']= c; pparams['LineColor'] = c;
        pparams['LineSize'] = 2
        pparams['Options']='l'
        pparams['isTH1']=False
        aplot = Plot(Name=hist.GetName(), Histo=hist,**pparams)
        aplot.normalize = False ;aplot.scaleFactor = None
#             aplot.normalize = True ;aplot.scaleFactor = 1.0*xSections[tag]
        plots[-1].addPlot(aplot)

    
   
    canvas = []
    for plot in plots:
        canvas.append(plot.plot())                     